<a href="https://colab.research.google.com/github/RafaelCaballero/charla/blob/main/textos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Análisis de textos

Empezamos instalando una biblioteca que vamos a utilizar

In [ ]:
import warnings
warnings.filterwarnings('ignore')

bibliotecas = ["flair","goose3","wordcloud"]

import sys
import os.path
from subprocess import check_call
import importlib
import os
import numpy as np

def instala_bibliotecas(bibliotecas):
  print("Instalando bibliotecas")
  alguna = False
  for m in bibliotecas:
      # para el import quitamos [...] y ==...
      p = m.find("[")
      mi = m if p==-1 else m[:p]
      p = mi.find("==")
      mi = mi if p==-1 else mi[:p]

      torch_loader = importlib.util.find_spec(mi)
      if torch_loader is not None:
          print(m, " encontrado")
      else:
          print(m, " No encontrado, instalando...",end="")
          try:
            r = check_call([sys.executable, "-m", "pip", "install", "--user", m])
            print("¡Hecho!")
            alguna = True
          except:
            print("¡Problema al instalar ",m,"! ¿seguro que el módulo existe?",sep="")

  print("¡Terminado!")
  if alguna:
    print("Recuerda reiniciar (Entorno de ejecución + Reiniciar entorno de ejecución) para que la(s) nueva(s) biblioteca(s) estén disponbles")





instala_bibliotecas(bibliotecas)




### ¿De dónde salen los datos?



*   Programación con bibliotecas
*   [Programas especializados](https://chrome.google.com/webstore/detail/aliexpress-reviews-export/glnajkhggmbljlcafknlkcihcheoankc?hl=en)
* Bases de datos

In [ ]:
# origen https://www.amazon.com/-/es/product-reviews/B07ZPKF8RG/ref=cm_cr_unknown?ie=UTF8&filterByStar=two_star&reviewerType=all_reviews&pageNumber=1#reviews-filter-bar
textos = ['Broken. It was supposed to be a B-Day gift for my mom and now she doesn’t get a gift on her Bday',
          'Touchscreen became almost completely unresponsive over 20% of surface area within days',
          "A little over 90 days, hardware failure, NO Solution, DON'T BUY",
          "Wanted to love it, but it had too many problems upon arrival",
          "Phone is great but the battery health is 76%",
          "Happy customer! It came with protective glass installed.",
          "Great phone for the price!",
          "I love this phone",
          "Definitely Recommend!"]

Librería Flair

In [ ]:
from flair.models import TextClassifier
from flair.data import Sentence
classifier = TextClassifier.load('en-sentiment')

for s in textos:
  text = Sentence(s)

  classifier.predict(text)
  value = text.labels[0].to_dict()
  print(s)
  print(value)
  print("="*30)

Para jugar

In [ ]:
s = "Pessimism is my default setting."
text = Sentence(s)
classifier = TextClassifier.load('en-sentiment')
classifier.predict(text)
value = text.labels[0].to_dict()
print(value)

## NER (Named Entity Recognition)

Reconocimiento de sustantivos que denotan nombres, organizaciones, etc. Importante para saber de qué se está hablando en textos complejos

In [ ]:
from flair.data import Sentence
from flair.models import SequenceTagger

# load tagger
tagger = SequenceTagger.load("flair/ner-spanish-large")

In [ ]:


# make example sentence
sentence = Sentence("George Washington fue a Washington, mientras que mi amiga Belén visitaba Belén")

# predict NER tags
tagger.predict(sentence)

# print sentence
print(sentence)

# print predicted NER spans
print('The following NER tags are found:')
# iterate over entities and print
for entity in sentence.get_spans('ner'):
    print(entity)

In [ ]:
s = """
Hermoso y clásico apartamento a cuadras de la Plaza Mayor
10
Liked · Lindo apartamento. Cómodo y espacioso. Muy bien ubicado. Javier fue muy amable en todo. Pasamos una muy grata estancia como familia.
Disliked · La señal Wifi tenía bajo rango, desconectandose cada cierto tiempo
"""


sentence = Sentence(s)

# predict NER tags
tagger.predict(sentence)


# print predicted NER spans
print('The following NER tags are found:')
# iterate over entities and print
for entity in sentence.get_spans('ner'):
    print(entity)

## Nubes de palabras

In [ ]:
from goose3 import Goose
import spacy
from wordcloud import WordCloud
import matplotlib.pyplot as plt

# descargamos la noticia
g = Goose()
pag_dir = "https://elpais.com/diario/1980/12/10/cultura/345250806_850215.html"
#pag_dir = "https://elpais.com/espana/2023-02-23/espana-sufre-hoy-el-dia-mas-duro-del-nuevo-episodio-invernal-con-11-comunidades-en-alerta-por-nevadas-a-cotas-muy-bajas.html"

articulo = g.extract(url=pag_dir)
texto = articulo.cleaned_text

# generar nube de palabras
nlp = spacy.blank('es')
palabrasvacías = nlp.Defaults.stop_words
palabrasvacías.add("zona")
palabrasvacías.add("años")
w  = WordCloud(stopwords = palabrasvacías)
w.generate(texto)

# mostramos el resultado
fig, ax = plt.subplots(1,1, figsize=(20,7),dpi=100)
plt.imshow(w)
fig.savefig("wordcloud1.png")